# MRP Knockout
```
pi:ababaian
files: ~/Crown/data2/MRPko
start: 2017 08 24
complete : 2017 08 29
```

## Introduction

In the CRC libraries, RNase MRP, (RMRP or MRP RNA) segregates MACP normal and hypo-MACP libraries from differential gene expression analysis. [Goldfarb](http://genesdev.cshlp.org/content/31/1/59.full) knocked out the gene in HeLa cells. They also show that knockout of MRP leads to pre-rRNA accumulation so it's possible that the hypo-MACP phenotype is a result of massive pre-rRNA accumulation and the readout in RNAseq.


#### RMRP in CRC30 DGE
![RMRP in CRC30](../../data2/crcmod_dge/plot/RMRP_crc30.png)

### Hypothesis
- MRP knockout will result in the accumulation of pre-rRNA and an increase in 1248U Reference Allele Frequency as seen in the CRC libraries.



## Materials and Methods
Analysis performed on GSC systems

Prefetch from SRA. hgr1 pipeline. adCalc for frequency.

In [ ]:
# prefetch
prefetch SRR5112458
prefetch SRR5112459
prefetch SRR5112460
prefetch SRR5112461
prefetch SRR5112462
prefetch SRR5112463
# fastq-dump
fastq-dump --gzip --split-files SRR5112458
fastq-dump --gzip --split-files SRR5112459
fastq-dump --gzip --split-files SRR5112460
fastq-dump --gzip --split-files SRR5112461
fastq-dump --gzip --split-files SRR5112462
fastq-dump --gzip --split-files SRR5112463

In [ ]:
#!/bin/bash
# mrpko_align_hgr1.fa
# rDNA alignment pipeline
# for MRP KO data on GSC to hgr1
# 170614 -- 2156 build
# xhost10 

# Control Panel -------------------------------

# Project Dir
  BASE='/home/ababaian/projects/MRPko'
  cd $BASE

# Sequencing Data
  CRC_DIR='/home/ababaian/projects/MRPko'
  LIB_LIST='mrpko_data.txt' # list of crc data fastq files
  
# CPU
  THREADS='2'
  
# Initialize start-up sequence ----------------
# Make working directory
  mkdir -p align

#Resources
  aws s3 cp s3://crownproject/resources/hgr1.fa ./
  samtools faidx hgr1.fa
  bowtie2-build hgr1.fa hgr1


# ---------------------------------------------
# SCRIPT LOOP ---------------------------------
# ---------------------------------------------
# For each line in input LIB_LIST; run the pipeline

cat $LIB_LIST | while read LINE
do
    #Initialize Run
    echo "Start Iteration:"
    echo "  $LINE"
    echo ''
    
    LIBRARY=$(echo $LINE | cut -f1 -d' ' -) # Library Name
    RGSM=$(echo $LINE | cut -f2 -d' ' -)    # Sample / Patient Identifer
    RGID=$(echo $LINE | cut -f3 -d' ' -)    # Read Group ID
    RGLB=$(echo $LINE | cut -f3 -d' ' -)    # Library Name. Accession Number
    RGPL='ILLUMINA'                   # Sequencing Platform.
    RGPO=$(echo $LINE | cut -f4 -d' '  -)    # Patient Population

    FASTQ1=$(echo $LINE | cut -f5 -d' ' -)  # Filename Read 1
    FASTQ2=$(echo $LINE | cut -f6 -d' ' -)  # Filename Read 2
    
    FQ1="$CRC_DIR/$FASTQ1"            # Fastq1 Filepath
    FQ2="$CRC_DIR/$FASTQ2"            # Fastq2 Filepath
    
    # Extract Sequencing Run Info
    RGPU=$RGID
    
    # Bowtie2: align to genome
    bowtie2 --very-sensitive-local -p $THREADS --rg-id $RGID \
      --rg LB:$RGLB --rg SM:$RGSM \
      --rg PL:$RGPL --rg PU:$RGPU \
      -x hgr1 -1 $FQ1 -2 $FQ2 |\
      samtools view -bS - > aligned_unsorted.bam
      

    # Sort the library
    samtools sort aligned_unsorted.bam $LIBRARY.hgr1.bam

    # Index the library
    samtools index $LIBRARY.hgr1.bam

    # Calculate flagstats for library
    samtools flagstat $LIBRARY.hgr1.bam > $LIBRARY.flagstat
    
    
    # Rename the total Bam Files
      rm aligned_unsorted.bam


done

# Primary VCF ----------------------------
# N/A

# Script complete

In [ ]:
#!/bin/bash
# ADcalc.sh
# Allelic Depth Calculator
# for a range of coordinates

# Controls -----------------
REGION='chr13:1004900-1005000'
OUTPUT='macp_MRPko.gvcf'
DEPTH='100000'
BAMLIST='bam.list'

# Iterate through every bam file in directory
# look-up position and return VCF

bcftools mpileup -f hgr1.fa \
 --max-depth $DEPTH --min-BQ 30 \
 -a FORMAT/DP,AD \
 -r $REGION \
 --ignore-RG \
 -b $BAMLIST |
 bcftools annotate -x INFO,FORMAT/PL - |
 bcftools view -O v -H - >> $OUTPUT


## Results

The output of the above analysis. macp_MRPko.gvcf. Which can really be boiled down to one line
```
chr13	1004908	.	T	C,A,G	0	.	.	DP:AD	18747:5996,11780,850,121	16838:5223,10661,852,102	14596:4999,8777,733,87	12405:4332,7383,622,68	18853:7218,10531,942,162	12588:4529,7359,616,84
```

Complete Output
```
chr13	1004900	.	A	C,G,T	0	.	.	DP:AD	70175:70170,4,1,0	73300:73297,3,0,0	74241:74238,3,0,0	75798:75786,9,1,2	70741:70736,2,3,0	77571:77566,4,1,0
chr13	1004901	.	A	G,C,T	0	.	.	DP:AD	66671:66671,0,0,0	70197:70197,0,0,0	70945:70943,0,1,1	72699:72696,1,2,0	67345:67345,0,0,0	74634:74630,3,0,1
chr13	1004902	.	T	C,A,G	0	.	.	DP:AD	4626:4626,0,0,0	17686:17681,4,1,0	20298:20293,3,1,1	25195:25188,6,0,1	3846:3846,0,0,0	30774:30766,8,0,0
chr13	1004903	.	T	C,A,G	0	.	.	DP:AD	5430:5430,0,0,0	18217:18182,34,1,0	20713:20635,76,1,1	25497:25328,165,4,0	4751:4751,0,0,0	31105:30836,261,4,4
chr13	1004904	.	T	C,A,G	0	.	.	DP:AD	8351:8351,0,0,0	20075:20017,53,4,1	22046:21878,158,10,0	26355:25995,337,20,3	7332:7332,0,0,0	31802:31118,653,24,7
chr13	1004905	.	G	<*>	0	.	.	DP:AD	14771:14771,0	20824:20824,0	21238:21238,0	25351:25351,0	14398:14398,0	30135:30135,0
chr13	1004906	.	A	C,<*>	0	.	.	DP:AD	12989:12987,2,0	15115:15111,4,0	15254:15253,1,0	18786:18785,1,0	13240:13227,13,0	23406:23404,2,0
chr13	1004907	.	C	T,A,<*>	0	.	.	DP:AD	17040:17038,2,0,0	17259:17259,0,0,0	16936:16934,2,0,0	19711:19710,1,0,0	16950:16947,2,1,0	23653:23652,1,0,0
chr13	1004908	.	T	C,A,G	0	.	.	DP:AD	18747:5996,11780,850,121	16838:5223,10661,852,102	14596:4999,8777,733,87	12405:4332,7383,622,68	18853:7218,10531,942,162	12588:4529,7359,616,84
chr13	1004909	.	C	T,G,A	0	.	.	DP:AD	22990:22987,3,0,0	20806:20804,2,0,0	17974:17971,3,0,0	15360:15359,0,1,0	23662:23660,1,0,1	15831:15829,2,0,0
chr13	1004910	.	A	C,G,T	0	.	.	DP:AD	32009:32005,1,3,0	29338:29336,1,0,1	24653:24653,0,0,0	21303:21302,1,0,0	32578:32571,4,2,1	22006:22006,0,0,0
chr13	1004911	.	A	G,T,C	0	.	.	DP:AD	33404:33403,1,0,0	30912:30910,1,0,1	25634:25633,1,0,0	22104:22102,2,0,0	33521:33520,0,1,0	22482:22481,0,1,0
chr13	1004912	.	C	T,G,A	0	.	.	DP:AD	35153:35151,1,0,1	32846:32839,5,2,0	27509:27503,0,3,3	23350:23349,1,0,0	35605:35601,2,2,0	23439:23437,2,0,0
chr13	1004913	.	A	G,T,C	0	.	.	DP:AD	39086:39075,7,1,3	36514:36509,2,2,1	30566:30558,3,3,2	25663:25659,1,1,2	39463:39455,3,4,1	25831:25823,5,1,2
chr13	1004914	.	C	T,G,A	0	.	.	DP:AD	61048:61032,9,6,1	57269:57261,3,2,3	46943:46934,6,1,2	41241:41232,5,3,1	63839:63824,11,3,1	41926:41918,5,1,2
chr13	1004915	.	G	A,T,C	0	.	.	DP:AD	80936:80928,7,1,0	81157:81151,3,1,2	81010:81007,1,1,1	80673:80665,6,1,1	82382:82379,2,0,1	82011:82008,2,1,0
chr13	1004916	.	G	A,T,C	0	.	.	DP:AD	84505:84490,7,5,3	85339:85323,6,6,4	84023:84010,5,4,4	83211:83204,2,1,4	85178:85162,7,8,1	84199:84189,7,2,1
chr13	1004917	.	G	A,C,T	0	.	.	DP:AD	82366:81550,811,5,0	83947:83250,689,8,0	81785:81169,609,6,1	80515:80037,470,8,0	82717:81895,811,11,0	81304:80839,459,6,0
chr13	1004918	.	A	C,T,G	0	.	.	DP:AD	80713:80689,18,2,4	81958:81927,26,3,2	79331:79313,14,1,3	77501:77486,13,2,0	80456:80425,23,6,2	78688:78667,15,3,3
chr13	1004919	.	A	C,G,T	0	.	.	DP:AD	75686:75660,16,8,2	77662:77636,23,1,2	73407:73392,8,3,4	70833:70811,16,4,2	75721:75692,23,6,0	72121:72090,19,10,2
chr13	1004920	.	A	C,T,G	0	.	.	DP:AD	66665:66614,23,20,8	69977:69944,19,11,3	64128:64096,15,9,8	60204:60168,16,11,9	66867:66823,27,12,5	61700:61665,19,10,6
chr13	1004921	.	C	T,A,G	0	.	.	DP:AD	84248:84228,12,1,7	85223:85210,7,5,1	83075:83059,9,4,3	82046:82032,9,3,2	84139:84124,8,5,2	82663:82653,6,3,1
chr13	1004922	.	C	T,A,G	0	.	.	DP:AD	81074:81056,17,0,1	82554:82536,10,6,2	79698:79682,9,3,4	77972:77954,9,5,4	80966:80945,8,6,7	78463:78444,11,4,4
chr13	1004923	.	T	G,C,A	0	.	.	DP:AD	74330:74323,4,2,1	76178:76151,20,5,2	72041:72018,15,2,6	69586:69575,6,5,0	73769:73513,253,2,1	69765:69747,12,4,2
chr13	1004924	.	C	T,A,G	0	.	.	DP:AD	58780:58769,7,4,0	62415:62405,5,3,2	55275:55267,6,2,0	51031:51026,5,0,0	58248:58228,13,6,1	50469:50460,5,3,1
chr13	1004925	.	A	C,G,T	0	.	.	DP:AD	51150:51062,75,10,3	55401:55327,64,7,3	46741:46666,68,6,1	42928:42871,52,3,2	50352:50248,94,7,3	41646:41565,71,4,6
chr13	1004926	.	C	T,A,G	0	.	.	DP:AD	81698:81672,16,6,4	83063:83035,20,4,4	80495:80467,23,2,3	80132:80111,15,2,4	82126:82102,21,2,1	80481:80460,14,4,3
chr13	1004927	.	C	T,A,G	0	.	.	DP:AD	82023:82009,10,3,1	83040:83025,11,0,4	80542:80528,11,3,0	80138:80122,10,6,0	81807:81789,12,5,1	80444:80430,11,2,1
chr13	1004928	.	C	T,G,A	0	.	.	DP:AD	81465:81447,9,4,5	82924:82907,10,5,2	80730:80705,17,5,3	80312:80289,17,4,2	81520:81504,13,2,1	80599:80580,16,2,1
chr13	1004929	.	G	A,C,T	0	.	.	DP:AD	78838:78826,6,4,2	80872:80864,7,0,1	78933:78926,5,1,1	78820:78808,6,4,2	79519:79509,7,1,2	79107:79104,2,1,0
chr13	1004930	.	G	A,C,T	0	.	.	DP:AD	79331:79319,4,5,3	81141:81129,5,5,2	78393:78382,4,3,4	77455:77447,6,1,1	79526:79513,4,7,2	77864:77850,6,6,2
chr13	1004931	.	C	T,G,A	0	.	.	DP:AD	80463:80450,8,2,3	82053:82031,16,3,3	78676:78648,20,6,2	78717:78697,17,0,3	80457:80441,10,4,2	79098:79079,12,5,2
chr13	1004932	.	C	T,A,G	0	.	.	DP:AD	81818:81799,14,2,3	84135:84114,18,3,0	80833:80809,17,6,1	80453:80437,13,3,0	82517:82494,20,3,0	80563:80545,16,1,1
chr13	1004933	.	C	T,G,A	0	.	.	DP:AD	85777:85756,12,5,4	87103:87083,10,5,5	85711:85680,17,10,4	85481:85455,18,4,4	85878:85860,13,2,3	85688:85658,19,8,3
chr13	1004934	.	G	A,T,C	0	.	.	DP:AD	84897:84877,10,3,7	86143:86128,6,7,2	85558:85546,6,3,3	85487:85467,11,6,3	85247:85234,10,0,3	85797:85780,5,6,6
chr13	1004935	.	G	A,C,T	0	.	.	DP:AD	83909:83862,40,4,3	84918:84883,26,9,0	84124:84068,43,7,6	83821:83747,63,5,6	84076:84047,25,2,2	84197:84161,22,11,3
chr13	1004936	.	A	C,G,T	0	.	.	DP:AD	76247:76054,174,10,9	77600:77433,153,8,6	76930:76756,158,8,8	76622:76502,110,7,3	76572:76376,178,10,8	77039:76879,145,10,5
chr13	1004937	.	C	T,A,G	0	.	.	DP:AD	81668:81633,25,8,2	82628:82602,21,3,2	81535:81505,24,4,2	80775:80744,28,1,2	81691:81672,12,5,2	81236:81215,15,3,3
chr13	1004938	.	A	C,T,G	0	.	.	DP:AD	78751:78582,147,15,7	80159:80018,124,14,3	77986:77825,133,22,6	76524:76418,92,11,3	79043:78887,134,19,3	76842:76702,124,9,7
chr13	1004939	.	C	T,G,A	0	.	.	DP:AD	85693:85664,15,13,1	86268:86244,15,8,1	83357:83330,18,6,3	81455:81435,14,3,3	85475:85453,17,3,2	81909:81892,12,4,1
chr13	1004940	.	G	A,T,C	0	.	.	DP:AD	85173:85158,6,9,0	85654:85645,7,2,0	84934:84919,9,4,2	84048:84028,13,5,2	84848:84833,7,3,5	84716:84705,5,4,2
chr13	1004941	.	G	A,T,C	0	.	.	DP:AD	84970:84956,7,6,1	85623:85608,7,3,5	84713:84705,4,1,3	83929:83915,8,3,3	84706:84689,6,6,5	84428:84412,5,5,6
chr13	1004942	.	A	C,T,G	0	.	.	DP:AD	79850:79768,56,12,14	80549:80497,34,12,6	80092:80036,39,11,6	79566:79526,23,9,8	79672:79618,42,7,5	79892:79831,42,10,9
chr13	1004943	.	C	T,A,G	0	.	.	DP:AD	82993:82963,25,2,3	83812:83787,18,7,0	82946:82923,21,2,0	82359:82331,19,8,1	82739:82713,20,4,2	83084:83062,18,3,1
chr13	1004944	.	A	C,G,T	0	.	.	DP:AD	82743:82705,14,14,10	82936:82904,12,14,6	83273:83218,21,20,14	81701:81673,7,11,10	82275:82237,18,13,7	83506:83469,20,10,7
chr13	1004945	.	G	T,A,C	0	.	.	DP:AD	85065:84542,518,3,2	85657:85195,460,1,1	85444:84993,439,6,6	83143:82644,488,4,7	84906:84462,441,3,0	85787:85332,446,7,2
chr13	1004946	.	G	T,A,C	0	.	.	DP:AD	85123:85112,6,4,1	85702:85694,2,4,2	84693:84683,6,0,4	81312:81304,1,4,3	84615:84610,4,1,0	84596:84578,7,6,5
chr13	1004947	.	A	T,C,G	0	.	.	DP:AD	80402:80324,53,11,14	80541:80479,44,10,8	80945:80877,48,10,10	80643:80590,41,7,5	80245:80186,40,16,3	81325:81258,41,15,11
chr13	1004948	.	T	G,C,A	0	.	.	DP:AD	80349:80317,14,12,6	80626:80571,39,14,2	80588:80542,26,16,4	80357:80318,14,20,5	80003:79657,336,8,2	80765:80706,32,19,8
chr13	1004949	.	T	G,C,A	0	.	.	DP:AD	84022:83992,13,13,4	84008:83959,39,7,3	83902:83869,24,9,0	82970:82944,12,10,4	83427:83079,342,6,0	83725:83680,28,13,4
chr13	1004950	.	G	C,A,T	0	.	.	DP:AD	82491:82480,2,5,4	83110:83101,6,1,2	83269:83252,11,4,2	82200:82185,7,6,2	82251:82239,4,6,2	83415:83406,4,5,0
chr13	1004951	.	A	C,T,G	0	.	.	DP:AD	79433:79386,33,10,4	80035:79995,21,10,9	79973:79925,22,13,13	78424:78382,23,7,12	79337:79285,37,9,6	79872:79835,29,3,5
chr13	1004952	.	C	T,A,G	0	.	.	DP:AD	85236:85221,11,2,2	85517:85503,9,4,1	85649:85623,17,4,5	85171:85157,7,4,3	84713:84689,22,0,2	85751:85735,11,3,2
chr13	1004953	.	A	C,T,G	0	.	.	DP:AD	83922:83840,65,10,7	84044:83976,51,13,4	84587:84505,59,14,9	84003:83945,39,9,10	83509:83417,78,4,10	84739:84675,50,7,7
chr13	1004954	.	G	C,A,T	0	.	.	DP:AD	85436:85413,8,9,6	85913:85898,7,6,2	86163:86144,10,6,3	85281:85258,10,7,6	85175:85162,7,3,3	86251:86237,5,6,3
chr13	1004955	.	A	C,T,G	0	.	.	DP:AD	82699:82614,61,18,6	82546:82481,46,14,5	83401:83326,57,13,5	82858:82797,41,9,11	82171:82085,76,7,3	83587:83516,46,21,4
chr13	1004956	.	T	G,C,A	0	.	.	DP:AD	81677:81666,3,6,2	81540:81518,15,5,2	82194:82182,8,2,2	81105:81090,6,8,1	80912:80861,44,2,5	82815:82804,7,4,0
chr13	1004957	.	T	G,C,A	0	.	.	DP:AD	84373:84362,5,5,1	84213:84190,15,7,1	84360:84330,19,10,1	83663:83644,8,10,1	83655:83592,58,5,0	84513:84484,19,8,2
chr13	1004958	.	G	T,C,A	0	.	.	DP:AD	81504:81491,4,7,2	81702:81691,5,0,6	82736:82719,8,6,3	82125:82116,6,0,3	81263:81250,7,2,4	83124:83111,2,9,2
chr13	1004959	.	A	C,T,G	0	.	.	DP:AD	82157:82000,139,6,12	82193:82032,141,14,6	82783:82642,125,11,5	82260:82142,102,12,4	81793:81632,143,15,3	83209:83073,123,11,2
chr13	1004960	.	T	G,C,A	0	.	.	DP:AD	82281:82269,4,4,4	82377:82357,11,6,3	82656:82641,4,8,3	82370:82354,7,5,4	81744:81680,54,6,4	82833:82817,11,4,1
chr13	1004961	.	A	C,T,G	0	.	.	DP:AD	81996:81791,193,7,5	82332:82136,178,10,8	82745:82564,176,4,1	82102:81958,132,8,4	81793:81582,203,6,2	82920:82754,153,8,5
chr13	1004962	.	G	C,T,A	0	.	.	DP:AD	82229:82219,6,1,3	82579:82571,5,2,1	82891:82879,10,1,1	82614:82602,6,3,3	81978:81972,1,5,0	82971:82964,6,1,0
chr13	1004963	.	C	T,A,G	0	.	.	DP:AD	82055:82045,8,0,2	82078:82070,5,3,0	82304:82297,4,2,1	81679:81660,14,3,2	81028:81004,20,3,1	82277:82255,20,2,0
chr13	1004964	.	T	C,G,A	0	.	.	DP:AD	83512:83499,10,1,2	83419:83411,6,2,0	83655:83643,10,2,0	82536:82523,8,3,2	82326:82286,14,26,0	83522:83512,7,2,1
chr13	1004965	.	C	T,G,A	0	.	.	DP:AD	85288:85170,111,4,3	85148:84997,148,2,1	85175:85029,143,1,2	84901:84768,126,4,3	84617:84426,185,2,4	85511:85373,135,2,1
chr13	1004966	.	T	C,G,A	0	.	.	DP:AD	85283:85259,20,2,2	85255:85232,22,1,0	85699:85660,39,0,0	85501:85478,19,3,1	84525:84468,33,20,4	85988:85961,23,4,0
chr13	1004967	.	T	C,G,A	0	.	.	DP:AD	86284:86269,11,4,0	86113:86100,6,5,2	86059:86044,11,4,0	85118:85108,10,0,0	85476:85439,10,24,3	85977:85967,8,2,0
chr13	1004968	.	T	G,C,A	0	.	.	DP:AD	84344:84333,4,6,1	84824:84815,4,4,1	84342:84330,4,7,1	82181:82172,7,2,0	84302:84272,22,5,3	83102:83090,4,8,0
chr13	1004969	.	C	T,G,A	0	.	.	DP:AD	87389:87370,9,7,3	87168:87156,8,2,2	87319:87291,19,2,7	86680:86657,14,6,3	86521:86501,11,6,3	87391:87374,9,4,4
chr13	1004970	.	T	C,G,A	0	.	.	DP:AD	84789:84775,10,3,1	85254:85242,6,2,4	85306:85292,5,3,6	84640:84628,9,3,0	84887:84856,9,21,1	84719:84706,8,3,2
chr13	1004971	.	C	T,G,A	0	.	.	DP:AD	87662:87635,24,2,1	87439:87411,22,2,4	87813:87783,23,5,2	87456:87438,13,2,3	86939:86910,24,4,1	88161:88127,29,2,3
chr13	1004972	.	G	A,C,T	0	.	.	DP:AD	85177:85167,4,5,1	84965:84956,3,3,3	85680:85672,4,2,2	84852:84838,5,7,2	84619:84606,8,1,4	85851:85836,10,5,0
chr13	1004973	.	A	C,T,G	0	.	.	DP:AD	81161:81110,43,5,3	80717:80679,30,6,2	82234:82176,43,12,3	82345:82314,26,2,3	80319:80257,45,16,1	82927:82877,40,6,4
chr13	1004974	.	T	C,G,A	0	.	.	DP:AD	85686:85678,6,2,0	84851:84841,5,4,1	86200:86187,4,5,4	86323:86316,5,2,0	84387:84364,5,17,1	86839:86830,7,2,0
chr13	1004975	.	T	G,C,A	0	.	.	DP:AD	85367:85355,3,7,2	85299:85290,4,5,0	85276:85265,7,3,1	85124:85109,5,8,2	84590:84573,16,0,1	85370:85358,4,7,1
chr13	1004976	.	C	T,A,G	0	.	.	DP:AD	87447:87432,10,2,3	87227:87210,13,2,2	88180:88163,14,0,3	88604:88582,16,6,0	86538:86527,10,1,0	88831:88816,11,1,3
chr13	1004977	.	C	T,A,G	0	.	.	DP:AD	87898:87867,21,5,5	87505:87478,22,2,3	88088:88057,24,4,3	88754:88735,15,3,1	86908:86881,23,1,3	88641:88609,27,4,1
chr13	1004978	.	G	T,C,A	0	.	.	DP:AD	85898:85877,7,6,8	85995:85978,6,5,6	87188:87170,9,5,4	87687:87664,12,4,7	85459:85447,0,10,2	87729:87713,4,5,7
chr13	1004979	.	T	G,C,A	0	.	.	DP:AD	79549:79513,24,10,2	78587:78551,28,8,0	80937:80885,37,13,2	81159:81116,30,9,4	78647:78595,47,2,3	82208:82173,21,12,2
chr13	1004980	.	G	A,T,C	0	.	.	DP:AD	83205:83201,0,4,0	82641:82630,5,2,4	84723:84712,5,2,4	85053:85039,6,4,4	82501:82488,4,4,5	85963:85956,4,3,0
chr13	1004981	.	G	T,A,C	0	.	.	DP:AD	80317:80300,10,4,3	79483:79469,8,2,4	82301:82280,9,10,2	82777:82765,5,4,3	79374:79361,3,7,3	83644:83630,4,7,3
chr13	1004982	.	G	T,C,A	0	.	.	DP:AD	80949:80942,2,4,1	80200:80187,4,7,2	82754:82741,6,5,2	82882:82871,4,5,2	80185:80176,7,0,2	83687:83677,4,2,4
chr13	1004983	.	T	G,C,A	0	.	.	DP:AD	54020:53989,23,6,2	50313:50283,23,4,3	58715:58674,31,8,2	60482:60451,23,4,4	53235:53182,43,6,4	62105:62078,18,8,1
chr13	1004984	.	G	C,A,T	0	.	.	DP:AD	65108:65093,5,4,6	62668:62654,8,4,2	69018:69006,6,3,3	70397:70380,7,6,4	63853:63834,3,8,8	71517:71497,7,8,5
chr13	1004985	.	G	C,T,A	0	.	.	DP:AD	74762:74700,47,12,3	73692:73637,33,13,9	77338:77289,37,9,3	77917:77857,46,10,4	73963:73909,48,4,2	78780:78706,62,5,7
chr13	1004986	.	T	G,C,A	0	.	.	DP:AD	56889:56831,41,15,2	53594:53538,42,13,1	61216:61147,51,13,5	62723:62653,53,15,2	56547:56446,77,23,1	64258:64187,48,20,3
chr13	1004987	.	G	C,T,A	0	.	.	DP:AD	74342:74313,9,10,10	73671:73644,11,12,4	76175:76152,11,5,7	76634:76611,13,5,5	73503:73473,14,12,4	77406:77376,14,6,10
chr13	1004988	.	G	T,A,C	0	.	.	DP:AD	81373:81355,8,8,2	80694:80683,3,4,4	82856:82851,1,2,2	82537:82518,12,4,3	80633:80622,1,5,5	83695:83679,6,7,3
chr13	1004989	.	T	G,C,A	0	.	.	DP:AD	58051:58012,24,10,5	55406:55371,28,7,0	62227:62173,38,9,7	63922:63878,31,10,3	57617:57549,51,15,2	65079:65036,29,9,5
chr13	1004990	.	G	A,T,C	0	.	.	DP:AD	75473:75457,8,3,5	75074:75058,14,2,0	77778:77757,12,4,5	78457:78442,10,3,2	74846:74837,4,3,2	78987:78978,5,3,1
chr13	1004991	.	C	T,A,G	0	.	.	DP:AD	81366:81279,67,15,5	81011:80945,47,14,5	82815:82734,61,15,5	83237:83169,50,13,5	80731:80660,50,12,9	83768:83690,56,14,8
chr13	1004992	.	A	G,C,T	0	.	.	DP:AD	77896:77614,156,84,42	77830:77603,126,74,27	80350:80117,109,78,46	80682:80484,101,67,30	77407:77169,136,65,37	81177:80965,105,65,42
chr13	1004993	.	T	C,G,A	0	.	.	DP:AD	77069:76909,134,17,9	77305:77163,108,19,15	79548:79403,96,38,11	80282:80172,76,26,8	76998:76847,111,34,6	81012:80873,100,33,6
chr13	1004994	.	G	A,C,T	0	.	.	DP:AD	78327:78245,77,4,1	78425:78357,61,3,4	81404:81325,64,11,4	82478:82413,55,9,1	78087:77991,85,8,3	83142:83079,56,6,1
chr13	1004995	.	G	A,T,C	0	.	.	DP:AD	76595:76541,48,3,3	76905:76865,26,8,6	79481:79449,21,4,7	80626:80595,26,3,2	76964:76925,28,5,6	81624:81589,24,6,5
chr13	1004996	.	C	T,A,G	0	.	.	DP:AD	68086:68073,8,4,1	69115:69105,8,1,1	72740:72721,7,5,7	74911:74897,10,1,3	68615:68603,9,2,1	75777:75756,13,6,2
chr13	1004997	.	C	T,G,A	0	.	.	DP:AD	77605:77550,54,1,0	78022:77965,51,4,2	80273:80209,61,3,0	81279:81198,78,3,0	77527:77476,47,3,1	81622:81556,63,2,1
chr13	1004998	.	G	A,T,C	0	.	.	DP:AD	74260:74249,6,5,0	74619:74611,5,2,1	77727:77713,7,6,1	78731:78716,7,3,5	73711:73696,10,3,2	78960:78951,4,2,3
chr13	1004999	.	T	C,G,A	0	.	.	DP:AD	64388:64381,5,1,1	63430:63426,3,1,0	68724:68717,4,2,1	70171:70164,6,1,0	62719:62717,1,0,1	70144:70141,1,2,0
chr13	1005000	.	T	C,A,G	0	.	.	DP:AD	72016:72008,6,1,1	71188:71182,2,3,1	73831:73827,3,0,1	74631:74623,6,1,1	71175:71165,9,1,0	75547:75543,1,1,2
```


## Results Analyzed

There is a hypo-modification phenotype in the knockout cells. It's not as extreme (upto 0.80 RAF) as in the CRC but can at least partially explain the contribution to macp-hypomodification. MRP_1 and MRP_4 are two different guide RNAS (each in duplicate) compared to the control knockout cells.

![MRPko_1248U](../../data2/MRPko/plot/MRPko_1248U.png)


### Discussion
    - The baseline RAF is quite low in this library at ~0.32. The particular RT used in these kits might be a good one to investigate for higher dynamic range in the macp-assay
    - The results partiall explain hypo-modification but not as complete as seen in the CRC libraries
    - Hypothesis is true, but again the effect size may not explain everything
    - Note this is a knockout of RPPH1, in hypo-mod CRC libraries those cells still have 'normal' levels of RPPH1, they simply fail to up-regulate it as seen in the 'control crc' libraries.

